## Primera entrega proyecto ETL

Autor: Alejandro Arteaga

In [1]:
import os
import zipfile

# Configurar API Key si no está en la carpeta correcta
os.environ['KAGGLE_CONFIG_DIR'] = "C:/Users/Alejandro Arteaga/Downloads"

# Descargar el dataset
!kaggle datasets download -d anandshaw2001/imdb-data


Dataset URL: https://www.kaggle.com/datasets/anandshaw2001/imdb-data
License(s): CC0-1.0
imdb-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
# Crea el path donde almacenar los datos extraidos

# Ruta del archivo ZIP descargado
zip_path = "C:/Users/Alejandro Arteaga/Desktop/ETL/Proyecto/imdb-data.zip"
extract_path = "C:/Users/Alejandro Arteaga/Desktop/ETL/Proyecto"


with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

### Carga de datos

In [3]:
import yaml
import psycopg2 
from psycopg2 import sql
from sqlalchemy import create_engine, text
import pandas as pd       

In [4]:
df = pd.read_csv("Imdb Movie Dataset.csv",sep = ",")
print(df)

             id                        title  vote_average  vote_count  \
0         27205                    Inception         8.364       34495   
1        157336                 Interstellar         8.417       32571   
2           155              The Dark Knight         8.512       30619   
3         19995                       Avatar         7.573       29815   
4         24428                 The Avengers         7.710       29166   
...         ...                          ...           ...         ...   
1048570  905156             鐨勯鏍肩殑椋庢牸儇妒呢蛹刹偎头         0.000           0   
1048571  905157             MILF & Cookies 3         0.000           0   
1048572  905158        The Choice of Staying         0.000           0   
1048573  905161  Luisa Schluckt Schon Wieder         0.000           0   
1048574  905163   Little White Slave Girls 8         0.000           0   

           status release_date     revenue  runtime  adult     budget  ...  \
0        Released    7/15/2010   

In [5]:
print(df.dtypes)
print(df.isnull().sum())

id                        int64
title                    object
vote_average            float64
vote_count                int64
status                   object
release_date             object
revenue                   int64
runtime                   int64
adult                      bool
budget                    int64
imdb_id                  object
original_language        object
original_title           object
overview                 object
popularity              float64
tagline                  object
genres                   object
production_companies     object
production_countries     object
spoken_languages         object
keywords                 object
dtype: object
id                           0
title                       13
vote_average                 0
vote_count                   0
status                       0
release_date            181817
revenue                      0
runtime                      0
adult                        0
budget                       0
imdb

In [6]:
# Debido a la gran cantidad de datos se elminan filas para reducir la exigencias computacional

# Número de filas a eliminar
num_filas_a_eliminar = 500000  

# Seleccionar aleatoriamente las filas a eliminar
filas_a_eliminar = df.sample(n=num_filas_a_eliminar, random_state=42).index

# Eliminar las filas
df3 = df.drop(filas_a_eliminar).reset_index(drop=True)

print(df3)

            id                                     title  vote_average  \
0       157336                              Interstellar         8.417   
1        19995                                    Avatar         7.573   
2        24428                              The Avengers         7.710   
3       293660                                  Deadpool         7.606   
4       118340                   Guardians of the Galaxy         7.906   
...        ...                                       ...           ...   
548570  905122                   POV Teens From Russia 2         0.000   
548571  905154  M.I.L.T.F. (Mothers I'd Like To Fuck) 16         0.000   
548572  905155                          MILF & Cookies 4         0.000   
548573  905156                          鐨勯鏍肩殑椋庢牸儇妒呢蛹刹偎头         0.000   
548574  905157                          MILF & Cookies 3         0.000   

        vote_count    status release_date     revenue  runtime  adult  \
0            32571  Released    11/5/2

In [7]:
print(df3.dtypes)
print(df3.isnull().sum())

id                        int64
title                    object
vote_average            float64
vote_count                int64
status                   object
release_date             object
revenue                   int64
runtime                   int64
adult                      bool
budget                    int64
imdb_id                  object
original_language        object
original_title           object
overview                 object
popularity              float64
tagline                  object
genres                   object
production_companies     object
production_countries     object
spoken_languages         object
keywords                 object
dtype: object
id                           0
title                        6
vote_average                 0
vote_count                   0
status                       0
release_date             95093
revenue                      0
runtime                      0
adult                        0
budget                       0
imdb

In [8]:
def load_config(file_path="config.yaml"):
    with open(file_path, "r") as file:
        return yaml.safe_load(file)

In [9]:
config = load_config()
db_config = config["database"]

# Load credentials
db_user = db_config["user"]
db_password = db_config["password"]
db_host = db_config["host"]
db_port = db_config["port"]
db_name = db_config["name"]

# DB connection
conn = psycopg2.connect(
    dbname="postgres",
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port
)
conn.autocommit = True

In [10]:
db_name = "etl_proyecto"
try:
    with conn.cursor() as cur:
        cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(db_name)))
        print(f"Base de datos '{db_name}' creada exitosamente.")
except psycopg2.errors.DuplicateDatabase:
    print(f"La base de datos '{db_name}' ya existe.")

Base de datos 'etl_proyecto' creada exitosamente.


In [11]:
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS tabla_proyecto_ETL2 (
                
            id BIGINT,
            title VARCHAR(1000),
            vote_average FLOAT,
            vote_count BIGINT,
            status VARCHAR(255),
            release_date VARCHAR(1000),
            revenue BIGINT,
            runtime BIGINT,
            adult BOOL,       
            budget BIGINT,
            imdb_id VARCHAR(1000),
            original_language VARCHAR(1000),                    
            original_title VARCHAR(1000),
            overview VARCHAR(1000),
            popularity FLOAT,
            tagline VARCHAR(1000),
            genres VARCHAR(1000),
            production_companies VARCHAR(1000),
            production_countries VARCHAR(1000),
            spoken_languages VARCHAR(1000),
            keywords VARCHAR(1000)                              
                                    
        );
    """))

    conn.commit()  # Asegúrate de confirmar los cambios
    print("Tabla 'tabla_proyecto_ETL2' creada exitosamente en PostgreSQL.")

    

Tabla 'tabla_proyecto_ETL2' creada exitosamente en PostgreSQL.


In [12]:
df3.shape

(548575, 21)

In [13]:
print(df3.columns)

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'budget', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')


In [14]:
df3['keywords'] = df3['keywords'].astype(str).str[:1000]
df3['production_companies'] = df3['production_companies'].astype(str).str[:1000]
df3['overview'] = df3['overview'].astype(str).str[:1000]
df3['genres'] = df3['genres'].astype(str).str[:1000]
df3['production_countries'] = df3['production_countries'].astype(str).str[:1000]
df3['spoken_languages'] = df3['spoken_languages'].astype(str).str[:1000]






In [15]:
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

with engine.connect() as conn:  
        stmt = text("""
        INSERT INTO tabla_proyecto_ETL2 (id, title, vote_average, vote_count, status, release_date, revenue, runtime, adult, budget, imdb_id, original_language, original_title, overview, popularity, tagline, genres, production_companies, production_countries, spoken_languages, keywords)
        VALUES (:id, :title, :vote_average, :vote_count, :status, :release_date, :revenue, :runtime, :adult, :budget, :imdb_id, :original_language, :original_title, :overview, :popularity, :tagline, :genres, :production_companies, :production_countries, :spoken_languages, :keywords)
    """)
        conn.execute(stmt, df3.to_dict(orient="records"))  
        conn. commit()
        
       
print("Los datos se cargaron exitosamente")
    

Los datos se cargaron exitosamente


In [16]:
# Leer datos de la tabla
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM tabla_proyecto_ETL2;"))
    rows = result.fetchall()

# Mostrar los resultados
print("Datos en 'tabla_proyecto_ETL2':")
for row in rows[:5]:
    print(row)

Datos en 'tabla_proyecto_ETL2':
(157336, 'Interstellar', 8.417, 32571, 'Released', '11/5/2014', 701729206, 169, False, 165000000, 'tt0816692', 'en', 'Interstellar', 'The adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.', 140.241, 'Mankind was born on Earth. It was never meant to die here.', 'Adventure, Drama, Science Fiction', 'Legendary Pictures, Syncopy, Lynda Obst Productions', 'United Kingdom, United States of America', 'English', 'rescue, future, spacecraft, race against time, artificial intelligence (a.i.), nasa, time warp, dystopia, expedition, space travel, wormhole, famine, ... (128 characters truncated) ... curious, space adventure, time paradox, thoughtful, time-manipulation, father daughter relationship, 2060s, cornfield, time manipulation, complicated')
(19995, 'Avatar', 7.573, 29815, 'Released', '12/15/2009', 2923706026, 162, Fa

In [17]:
with engine.connect() as conn:
    df4 = pd.read_sql("SELECT * FROM tabla_proyecto_ETL2", conn)

In [18]:
df4

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,budget,...,original_language,original_title,overview,popularity,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,157336,Interstellar,8.417,32571,Released,11/5/2014,701729206,169,False,165000000,...,en,Interstellar,The adventures of a group of explorers who mak...,140.241,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
1,19995,Avatar,7.573,29815,Released,12/15/2009,2923706026,162,False,237000000,...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
2,24428,The Avengers,7.710,29166,Released,4/25/2012,1518815515,143,False,220000000,...,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
3,293660,Deadpool,7.606,28894,Released,2/9/2016,783100000,108,False,58000000,...,en,Deadpool,The origin story of former Special Forces oper...,72.735,Witness the beginning of a happy ending.,"Action, Adventure, Comedy","20th Century Fox, The Donners' Company, Genre ...",United States of America,English,"superhero, anti hero, mercenary, based on comi..."
4,118340,Guardians of the Galaxy,7.906,26638,Released,7/30/2014,772776600,121,False,170000000,...,en,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",33.255,All heroes start somewhere.,"Action, Science Fiction, Adventure",Marvel Studios,United States of America,English,"spacecraft, based on comic, space, orphan, adv..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548570,905122,POV Teens From Russia 2,0.000,0,Released,1/1/2012,0,135,True,0,...,en,POV Teens From Russia 2,Russian youngsters get fucked in all their tig...,0.600,NaN,nan,Evil Playgrounds,nan,English,nan
548571,905154,M.I.L.T.F. (Mothers I'd Like To Fuck) 16,0.000,0,Released,NaN,0,0,True,0,...,en,M.I.L.T.F. (Mothers I'd Like To Fuck) 16,nan,0.600,NaN,nan,nan,nan,nan,nan
548572,905155,MILF & Cookies 4,0.000,0,Released,NaN,0,0,True,0,...,en,MILF & Cookies 4,nan,0.600,NaN,nan,nan,nan,nan,nan
548573,905156,鐨勯鏍肩殑椋庢牸儇妒呢蛹刹偎头,0.000,0,Released,NaN,0,0,False,0,...,zh,鐨勯鏍肩殑椋庢牸儇妒呢蛹刹偎头,nan,0.600,NaN,nan,nan,nan,nan,nan
